# Plan for today

- Download price data of multiple stocks
- Compute Daily and Monthly returns
- Compute Correlations
- Compute Cumulative Returns
- Annualized Returns and Risk (std. deviation)

In [ ]:
# Installation of python modules

!pip install pandas
!pip install numpy
!pip install yfinance
!pip install matplotlib

In [ ]:
# @title
# Importing data manipulation modules
import pandas as pd
import numpy as np

# Importing data visulations modules
import matplotlib.pyplot as plt

# Importing finance modules
import yfinance as yf

# Download data

We will be using a free python api library called [y-finance](https://pypi.org/project/yfinance/) and is a popular open-source library that allows users to download historical market data from Yahoo Finance. It simplifies the process of accessing financial data for stocks, bonds, and other assets, providing both historical and real-time data for analysis, research, and modeling.

Although, it is great for experimentation it is too unreliable to be used in the actaul implementation of algos due to rate limiting, throttling and data inaccuracy.

In [ ]:
# Ticker list to download
tickers = 'SPY QQQ TLT AAPL MSFT GOOG AMZN NFLX NVDA MA ADBE' # Why would we use these instead of just company names?

# Download data from Yahoo-Finance
data = yf.download(
    tickers = tickers, # The stocks do we want to get
    interval = "1d", # The sample rate of the data
    start="2020-01-01", # From what point do we want to start getting stock data
    end="2024-04-30", # From what point do we want to stop getting stock data
    ignore_tz=True,
    auto_adjust=True, # Adjust all fields by splits and dividends
)

In [ ]:
# What data does y-finance give us?
data

In [ ]:
print(data.columns)

In [ ]:
# Using the inbuilt pandas function describe() to give a description of the Close data
data['Close'].describe()

In [ ]:
# Using the inbuilt pandas function describe() to give a description of the Close data
data['Volume'].describe()

As we are only interested interested in the closing price on the day we will select this parameter and create a new DataFrame

In [ ]:
# Copy the close data over to a new dataframe
df = data['Close'].copy()

In [ ]:
# Display our data using the head(num_of_values) functions
df.head(10)

In [ ]:
# Check to see how many null (no values) we have in our data (it should be 0 if y-finance has worked)
print(f"There are {df.isnull().sum().sum()} null values across the DataFrame")

# If we did have any rows with null in we would use the next line to remove them:
df[df.notnull().all(1)].head()




Now let's compute a very simple metric called daily returns

In [ ]:
# Using pandas inbuilt function pct_change to compute the daily returns for us
df = df.pct_change()
df.head()

In [ ]:
# Fill the first row with zero
df = df.fillna(0)

In [ ]:
df.head()

## Some context of the data

Benchmarks:
- SPY: S&P 500
- QQQ: Nasdaq-100
- TLT: Long term bond US treasury ETF

In [ ]:
benchmarks = ['SPY', "QQQ", "TLT"]
stocks = ['AAPL', 'ADBE', 'AMZN', 'GOOG', 'MA', 'MSFT', 'NFLX', 'NVDA']

# Compute Monthly returns

$$
\text{Monthly Return} = \left( \prod_{i=1}^{n} (1 + r_i) \right) - 1
$$

Where:
- rᵢ = percentage daily return for each day *i* in the month
- Π represents the product of all (1 + rᵢ) terms for the days in the month.


In [ ]:
# Resample the data, grouping it into months for the monthly returns to be computed
monthly_ret = df.resample('ME')

# Display the first value of every group (month) for the first 5 months-
monthly_ret.first()[0:5]

In [ ]:
# Apply our custom function to these months
monthly_ret = monthly_ret.apply(lambda x: (1 + x).prod() - 1)

# Display data
monthly_ret.head(12)

### Correlation Matrix

- Most stocks are correlated
- Stocks and TLT have a negative correlation

In [ ]:
# Calculate the correlations between all the monthly returns of stocks with eachother
correlations = monthly_ret.corr()

# ** Code to Display Heat Map **

plt.figure(figsize=(8, 6))
plt.imshow(correlations, cmap='coolwarm', interpolation='none', aspect='auto')
plt.colorbar()

# Add labels for the axes
tick_marks = np.arange(len(correlations.columns))
plt.xticks(tick_marks, correlations.columns, rotation=45)
plt.yticks(tick_marks, correlations.columns)

plt.title('Correlation Matrix')
plt.show()

## Case Study: Airbus vs Boeing

Lets now used what we have done over the course of this notebook to explore the relationship between Airbus and Boeing.

Tickers: 'BA' and 'EADSY'

In [ ]:
# Load in the data for both stocks:

tickers = ['BA', 'EADSY']

data = yf.download(
    tickers = tickers, # The stocks do we want to get
    interval = "1d", # The sample rate of the data
    start="2020-05-01", # From what point do we want to start getting stock data
    end="2024-04-30", # From what point do we want to stop getting stock data
    ignore_tz=True,
    auto_adjust=True, # Adjust all fields by splits and dividends
)

data.head()

In [ ]:
# Take the close values
df = data['Close'].copy()

In [ ]:
df.describe()

In [ ]:
# Compute Daily returns and fill NaN values
df = df.pct_change().fillna(0)

# Resample the data, grouping it into months for the monthly returns to be computed
monthly_ret = df.resample('ME')

# Apply our custom function to these months
monthly_ret = monthly_ret.apply(lambda x: (1 + x).prod() - 1)

# Display data
monthly_ret.head()

In [ ]:
# Let's see what the worse month was for both companies...

ba_min = monthly_ret['BA'].min()
ba_min_index = monthly_ret['BA'].idxmin()

print(f'The worse Boeing day was a {round((ba_min * 100), 2)}% decrese in Boeing Stock on {ba_min_index.month_name()} {ba_min_index.year}')

ab_min = monthly_ret['EADSY'].min()
ab_min_index = monthly_ret['EADSY'].idxmin()

print(f'The worse Airbus day was a {round((ab_min * 100), 2)}% decrese in Airbus Stock on {ab_min_index.month_name()} {ab_min_index.year}')

#### What happened in those months?

**In September 2022:** Boeing announced a $200 million settlement with the U.S. Securities and Exchange Commission (SEC) over charges that the company misled investors about the safety of the 737 Max.

**In June 2022:** Shares in aircraft manufacturer Airbus (AIR) posted the biggest fall on the Paris Bourse after the company issued a profit warning due to problems with its supply chains and asset write-downs. At 10:42 UK time, the share price was down 11.2% at EUR 132.84 (£112.27).


In [ ]:
# Calculate the final returns over the time period
cumulative_ret = (1 + monthly_ret).cumprod()

In [ ]:
# Import in a custom plotting library
import plotly.express as px

# Let's plot the monthly returns of both these stocks
px.line(cumulative_ret)

In [ ]:
# We can use the cumulative return values to estimate how much we would had made if we invested £100 in Airbus and Boeing from July 2020 to Feb 2024

invested = 100 # Let's say we invested £100
final_cumulative_ret = cumulative_ret.iloc[-1] # Get the final cumulative return value

stock_value_airbus = invested * final_cumulative_ret['EADSY']
stock_value_boeing = invested * final_cumulative_ret['BA']

print(f'Value of the orginal £100 pound investment if it had been invested in Airbus: £{round(stock_value_airbus, 2)}')
print(f'Value of the orginal £100 pound investment if it had been invested in Boeing: £{round(stock_value_boeing, 2)}')

### Risk Anaylsis
Let's have a look at the best ways for us to identify and calculate the risk of an asset. One way to do that is to look at the standard deviation of a stock over time.


In [ ]:
# Get the std of the monthly returns:
risk = monthly_ret.std()

print('The risk of investing in Airbus is: ', round(risk['EADSY'], 4))
print('The risk of investing in Boeing is: ', round(risk['BA'], 4))

We can see that the standard deviation of the monthly returns for Airbus is far lower than Boeing which is exactly what we would expect. Airbus is a less risky investment.

# Challenge:

In this notebook we have explore some data exploration technqiues and touched on some basic mathematical approaches for analysing stocks.

**Your task**: Is to find/choose two stocks that you like, and calculate the best time to buy/sell these specific stocks, and then further calculate when the best time would be to buy both stocks at the same time.